In [4]:
import pandas as pd

In [3]:
#caminho Dani
bruto = pd.read_csv()

TypeError: read_csv() missing 1 required positional argument: 'filepath_or_buffer'

In [ ]:
#caminho Isa
bruto = pd.read_csv()

In [ ]:
#caminho Marcos
bruto = pd.read_csv(r'C:\Users\marcos220057\OneDrive - CNPEM - Centro Nacional de Pesquisa em Energia e Materiais\2sem2022\Aprendizado de Máquina\Neutral_Graphene_Oxide_Nanoflake_Dataset.csv')

In [9]:
#caminho Paola
bruto = pd.read_csv(r'C:\JupyterLab\ML-2022-DIMP\Neutral_Graphene_Oxide_Nanoflake_Dataset.csv')